In [1]:
import sqlite3
import numpy as np


def create_database(num_players):
    # 连接到SQLite数据库
    conn = sqlite3.connect('game_database.db')
    cursor = conn.cursor()

    # 如果表存在就删除
    cursor.execute('DROP TABLE IF EXISTS utility')
    # 创建一个表来存储策略和效用值
    create_table_query = f'''
        CREATE TABLE IF NOT EXISTS utility (
            id TEXT PRIMARY KEY,
            {', '.join([f'player{i}_strategy INTEGER' for i in range(num_players)])},
            {', '.join([f'player{i}_u INTEGER' for i in range(num_players)])}
        )
    '''
    cursor.execute(create_table_query)

    # 提交更改并关闭连接
    conn.commit()
    conn.close()


def insert_data(utility_matrix):
    # 连接到SQLite数据库
    conn = sqlite3.connect('game_database.db')
    cursor = conn.cursor()

    # 插入一些测试数据
    num_players = utility_matrix.shape[-1]
    num_strategies = utility_matrix.shape[:-1]

    for indices in np.ndindex(*num_strategies):
        strategy_id = '-'.join(map(str, indices))
        values = (strategy_id,) + tuple(indices) + \
            tuple(utility_matrix[indices].tolist())

        insert_query = f'''
            INSERT INTO utility VALUES ({', '.join(['?'] * (1 + num_players * 2))})
        '''
        print(values)
        cursor.execute(insert_query, values)

    # 提交更改并关闭连接
    conn.commit()
    conn.close()


def get_utility_by_strategy_id(strategy_id):
    conn = sqlite3.connect('game_database.db')
    cursor = conn.cursor()

    cursor.execute('SELECT * FROM utility WHERE id = ?', (strategy_id,))
    result = cursor.fetchone()

    conn.close()

    return result

In [13]:
num_players = 2
create_database(num_players)

In [14]:
# Example utility matrix for a three-player game with different strategy numbers
num_strategies = [8, 8]
utility_matrix_shape = tuple(num_strategies + [num_players])
utility_matrix = np.random.randint(0, 101, size=utility_matrix_shape)

In [15]:
insert_data(utility_matrix)

('0-0', 0, 0, 36, 57)
('0-1', 0, 1, 26, 57)
('0-2', 0, 2, 17, 100)
('0-3', 0, 3, 66, 85)
('0-4', 0, 4, 27, 99)
('0-5', 0, 5, 11, 60)
('0-6', 0, 6, 46, 67)
('0-7', 0, 7, 28, 23)
('1-0', 1, 0, 1, 60)
('1-1', 1, 1, 32, 41)
('1-2', 1, 2, 64, 95)
('1-3', 1, 3, 48, 87)
('1-4', 1, 4, 24, 34)
('1-5', 1, 5, 2, 51)
('1-6', 1, 6, 5, 70)
('1-7', 1, 7, 31, 9)
('2-0', 2, 0, 64, 20)
('2-1', 2, 1, 100, 58)
('2-2', 2, 2, 19, 88)
('2-3', 2, 3, 57, 40)
('2-4', 2, 4, 91, 78)
('2-5', 2, 5, 98, 35)
('2-6', 2, 6, 95, 51)
('2-7', 2, 7, 73, 6)
('3-0', 3, 0, 47, 30)
('3-1', 3, 1, 44, 11)
('3-2', 3, 2, 95, 12)
('3-3', 3, 3, 55, 85)
('3-4', 3, 4, 95, 38)
('3-5', 3, 5, 54, 48)
('3-6', 3, 6, 76, 22)
('3-7', 3, 7, 33, 83)
('4-0', 4, 0, 51, 28)
('4-1', 4, 1, 42, 7)
('4-2', 4, 2, 3, 30)
('4-3', 4, 3, 72, 14)
('4-4', 4, 4, 93, 31)
('4-5', 4, 5, 33, 84)
('4-6', 4, 6, 53, 12)
('4-7', 4, 7, 44, 66)
('5-0', 5, 0, 97, 60)
('5-1', 5, 1, 30, 73)
('5-2', 5, 2, 45, 13)
('5-3', 5, 3, 41, 57)
('5-4', 5, 4, 63, 17)
('5-5', 5, 5, 1

In [17]:
# 查询特定策略ID的效用值
result = get_utility_by_strategy_id('1-2')
print("Utility for strategy_id '1-2-1':", result)

Utility for strategy_id '1-2-1': ('1-2', 1, 2, 64, 95)


In [25]:
import sqlite3
import numpy as np


def fetch_utility_by_player_and_strategy(player_index, strategy_indices):
    conn = sqlite3.connect('game_database.db')
    cursor = conn.cursor()

    # Use placeholders for strategy indices in the query
    placeholders = ', '.join(['?' for _ in strategy_indices])

    # Construct the query
    query = f'SELECT * FROM utility WHERE player{player_index}_strategy IN ({placeholders})'

    # Convert strategy_indices to tuple
    strategy_indices_tuple = tuple(strategy_indices)

    cursor.execute(query, strategy_indices_tuple)
    rows = cursor.fetchall()

    conn.close()

    return rows


def calculate_regret_for_player(player_index, current_strategy_indices, utility_matrix):
    num_players = utility_matrix.shape[-1] // 2
    num_strategies = utility_matrix.shape[:-1]

    strategies = num_strategies[player_index]
    other_players = list(set(range(num_players)) - {player_index})

    regret_values = []

    for indices in np.ndindex(*num_strategies):
        if indices[player_index] not in current_strategy_indices:
            continue

        current_utility = utility_matrix[indices][player_index]
        max_other_utility = max(utility_matrix[indices][other_players])

        regret_values.append(max_other_utility - current_utility)

    return regret_values

# Example: Assuming you have already inserted data into the database

# Fetch utility matrix from the database
# num_players = 3
# utility_matrix = fetch_utility_matrix_from_db(num_players)

# Calculate regret values for player 0 when player 1 uses strategy 0 and player 2 uses strategy 1
# player_0_regret_values = calculate_regret_for_player(0, [0], utility_matrix)

# print("Regret Values for Player 0:", player_0_regret_values)

In [26]:
# Example: Testing the function
# Assuming you have already inserted data into the database

# Fetch utility values for player 0 with strategies 0 and 1
result = fetch_utility_by_player_and_strategy(0, [0, 1])

print("Fetched Utility Values:")
print(result)

Fetched Utility Values:
[('0-0', 0, 0, 36, 57), ('0-1', 0, 1, 26, 57), ('0-2', 0, 2, 17, 100), ('0-3', 0, 3, 66, 85), ('0-4', 0, 4, 27, 99), ('0-5', 0, 5, 11, 60), ('0-6', 0, 6, 46, 67), ('0-7', 0, 7, 28, 23), ('1-0', 1, 0, 1, 60), ('1-1', 1, 1, 32, 41), ('1-2', 1, 2, 64, 95), ('1-3', 1, 3, 48, 87), ('1-4', 1, 4, 24, 34), ('1-5', 1, 5, 2, 51), ('1-6', 1, 6, 5, 70), ('1-7', 1, 7, 31, 9)]
